# Exp 04. 인공지능 작사가

### 라이브러리 불러오기

In [1]:
import os
import glob
import re 
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

### 데이터 로드

In [2]:
txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = [] # 문장들이 담길 list

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담는다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


### 데이터 전처리

In [3]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)
    sentence = sentence.strip()
    sentence = '<start> ' + sentence + ' <end>'
    return sentence

print(preprocess_sentence("This @_is %;;;sample  ^^   ★   △sentence."))

<start> this is sample sentence . <end>


In [4]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    preprocessed_sentence = preprocess_sentence(sentence)
    if len(preprocessed_sentence.split()) > 15: continue
    corpus.append(preprocessed_sentence)
        
corpus[:15]

['<start> now i ve heard there was a secret chord <end>',
 '<start> that david played , and it pleased the lord <end>',
 '<start> but you don t really care for music , do you ? <end>',
 '<start> it goes like this <end>',
 '<start> the fourth , the fifth <end>',
 '<start> the minor fall , the major lift <end>',
 '<start> the baffled king composing hallelujah hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah your faith was strong but you needed proof <end>',
 '<start> you saw her bathing on the roof <end>',
 '<start> her beauty and the moonlight overthrew her <end>',
 '<start> she tied you <end>',
 '<start> to a kitchen chair <end>',
 '<start> she broke your throne , and she cut your hair <end>']

### 토큰화

In [5]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=12000, 
                                                      filters=' ',
                                                      oov_token="<unk>")
    tokenizer.fit_on_texts(corpus)
    tensor = tokenizer.texts_to_sequences(corpus)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   50    4 ...    0    0    0]
 [   2   15 2967 ...    0    0    0]
 [   2   33    7 ...   46    3    0]
 ...
 [   2    4  118 ...    0    0    0]
 [   2  258  194 ...   12    3    0]
 [   2    7   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7fd434093340>


In [6]:
print(tensor[:3])
tensor.shape

[[   2   50    4   95  303   62   53    9  946 6263    3    0    0    0
     0]
 [   2   15 2967  871    5    8   11 5739    6  374    3    0    0    0
     0]
 [   2   33    7   40   16  164  288   28  333    5   48    7   46    3
     0]]


(156013, 15)

In [7]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])
    
    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : i
5 : ,
6 : the
7 : you
8 : and
9 : a
10 : to


### 데이터셋 분리

In [8]:
src_input = tensor[:, :-1]
tgt_input = tensor[:, 1:]

print(src_input[0])
print(tgt_input[0])

[   2   50    4   95  303   62   53    9  946 6263    3    0    0    0]
[  50    4   95  303   62   53    9  946 6263    3    0    0    0    0]


In [9]:
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input,
                                                          tgt_input,
                                                          test_size=0.2,
                                                          random_state=42)

print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)
print("Source Validation:", enc_val.shape)
print("Target Validation:", dec_val.shape)

Source Train: (124810, 14)
Target Train: (124810, 14)
Source Validation: (31203, 14)
Target Validation: (31203, 14)


### 하이퍼 파라미터 설정

In [10]:
BATCH_SIZE = 256
embedding_size = 256
hidden_size = 1024

### 단방향 모델 생성

In [11]:
class TextGenerator_one(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.drop1 = tf.keras.layers.Dropout(0.2)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.drop1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    

model = TextGenerator_one(tokenizer.num_words + 1, embedding_size , hidden_size)

print('done')

done


### 모델 학습

In [12]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)
model.fit(enc_train, dec_train, epochs=10, batch_size=BATCH_SIZE, validation_data=(enc_val, dec_val))

Epoch 1/10
488/488 [==============================] - 95s 181ms/step - loss: 3.5200 - val_loss: 3.1672
Epoch 2/10
488/488 [==============================] - 96s 196ms/step - loss: 3.0418 - val_loss: 2.9687
Epoch 3/10
488/488 [==============================] - 96s 196ms/step - loss: 2.8675 - val_loss: 2.8520
Epoch 4/10
488/488 [==============================] - 96s 197ms/step - loss: 2.7391 - val_loss: 2.7720
Epoch 5/10
488/488 [==============================] - 97s 198ms/step - loss: 2.6331 - val_loss: 2.7061
Epoch 6/10
488/488 [==============================] - 97s 198ms/step - loss: 2.5377 - val_loss: 2.6539
Epoch 7/10
488/488 [==============================] - 97s 198ms/step - loss: 2.4484 - val_loss: 2.6082
Epoch 8/10
488/488 [==============================] - 97s 198ms/step - loss: 2.3636 - val_loss: 2.5667
Epoch 9/10
488/488 [==============================] - 97s 200ms/step - loss: 2.2837 - val_loss: 2.5379
Epoch 10/10
488/488 [==============================] - 98s 201ms/step - l

### 문장 생성

In [13]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    while True:
        predict = model(test_tensor)
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [14]:
print(generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20))
print(generate_text(model, tokenizer, init_sentence="<start> he was a", max_len=20))
print(generate_text(model, tokenizer, init_sentence="<start> when i was", max_len=20))

<start> i love you , liberian girl <end> 
<start> he was a <unk> <end> 
<start> when i was a <unk> , i m a weirdo <end> 


### 양방향 모델

In [15]:
class TextGenerator_one(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(hidden_size, return_sequences=True))
        self.drop1 = tf.keras.layers.Dropout(0.2)
        self.rnn_2 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(hidden_size, return_sequences=True))
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.drop1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    

model = TextGenerator_one(tokenizer.num_words + 1, embedding_size , hidden_size)

print('done')

done


In [16]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)
model.fit(enc_train, dec_train, epochs=5, batch_size=BATCH_SIZE, validation_data=(enc_val, dec_val))

Epoch 1/5
488/488 [==============================] - 219s 433ms/step - loss: 1.8324 - val_loss: 0.3888
Epoch 2/5
488/488 [==============================] - 210s 431ms/step - loss: 0.2252 - val_loss: 0.1641
Epoch 3/5
488/488 [==============================] - 211s 433ms/step - loss: 0.0645 - val_loss: 0.1017
Epoch 4/5
488/488 [==============================] - 211s 433ms/step - loss: 0.0148 - val_loss: 0.0857
Epoch 5/5
488/488 [==============================] - 212s 434ms/step - loss: 0.0040 - val_loss: 0.0764


In [17]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    while True:
        predict = model(test_tensor)
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [18]:
print(generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20))
print(generate_text(model, tokenizer, init_sentence="<start> he was a", max_len=20))
print(generate_text(model, tokenizer, init_sentence="<start> when i was", max_len=20))

<start> i love <end> 
<start> he was a fiend <end> 
<start> when i was mere <end> 


### 결론
- 단방향과 양방향 LSTM을 적용해 학습.
- 양방향이 단방향에 비해 시간이 배로 걸렸지만 성능 차이는 월등하게 높게 나옴.
- 단방향은 학습이 정상적으로 되지않음